In [1]:
import sys
import sklearn
!pip install package scikit-learn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [7 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\vish8\AppData\Local\Temp\pip-install-ry4j0v7c\package_6d656e350c314535b3a2281d3e245ad5\setup.py", line 4
          print """
          ^^^^^^^^^
      SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)?
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


In [ ]:
#Salvando as imagens geradas
from pathlib import Path
plt.rc('font', size=12)
plt.rc('axes', labelsize=12)
plt.rc('axes', titlesize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)


